## Agregação por distrito

Nesse notebook é feita a agregação dos dados por distritos do município

In [1]:
import geopandas as gpd
from shapely.geometry import LineString
from utils.load_csv import load_csv
from utils.load_shp import load_shp

In [2]:
df_final = load_csv("df_final.csv")

distritos = load_shp("distritos.shp")

In [3]:
df_final["lon"] = df_final["ponto_inicial"].apply(lambda x: eval(x)[0])
df_final["lat"] = df_final["ponto_inicial"].apply(lambda x: eval(x)[1])

In [4]:
gdf = gpd.GeoDataFrame(df_final, geometry=gpd.points_from_xy(df_final.lon, df_final.lat), crs="EPSG:4326")

In [5]:
trajetos = gdf.groupby("id_onibus")["geometry"].apply(lambda x: LineString(x.tolist()))
trajetos = gpd.GeoDataFrame(trajetos, geometry="geometry", crs="EPSG:4326").reset_index()

In [6]:
trajetos = trajetos.to_crs(distritos.crs)

In [7]:
trajetos_distritos = gpd.sjoin(trajetos, distritos, how="inner", predicate="intersects")

trajetos_distritos

,id_onibus,geometry,index_right,cd_identif,cd_identi0,cd_distrit,nm_distrit,sg_distrit,tx_escala,sg_fonte_o,dt_criacao,dt_atualiz,cd_usuario,cd_tipo_di,qt_area_qu,qt_area_me
0,1001,"LINESTRING (335420.733 7397246.561, 335622.462...",91,8583399.0,25.0,56,PARI,PRI,1:5000,GEOGSG,2004-01-01,2025-07-05,None,199299399,2.730,2729940.52
0,1001,"LINESTRING (335420.733 7397246.561, 335622.462...",76,8583430.0,7.0,86,VILA GUILHERME,VGL,1:5000,GEOGSG,2004-01-01,2025-07-05,None,199299399,7.201,7200560.70
1,1002,"LINESTRING (335553.706 7400313.049, 335755.483...",76,8583430.0,7.0,86,VILA GUILHERME,VGL,1:5000,GEOGSG,2004-01-01,2025-07-05,None,199299399,7.201,7200560.70
1,1002,"LINESTRING (335553.706 7400313.049, 335755.483...",95,8583426.0,5.0,82,TUCURUVI,TUC,1:5000,GEOGSG,2004-01-01,2025-07-05,None,199299399,9.441,9441317.18
2,1003,"LINESTRING (334104.451 7399279.07, 334306.193 ...",69,8583413.0,5.0,70,SANTANA,STN,1:5000,GEOGSG,2004-01-01,2025-07-05,None,199299399,13.092,13091744.80
3,1004,"LINESTRING (331989.004 7397294.618, 332190.686...",39,8583449.0,9.0,9,BOM RETIRO,BRE,1:5000,GEOGSG,2004-01-01,2025-07-05,None,199299399,4.206,4205896.64
3,1004,"LINESTRING (331989.004 7397294.618, 332190.686...",69,8583413.0,5.0,70,SANTANA,STN,1:5000,GEOGSG,2004-01-01,2025-07-05,None,199299399,13.092,13091744.80
4,1005,"LINESTRING (335333.2 7396337.369, 335534.914 7...",52,8583450.0,25.0,10,BRAS,BRS,1:5000,GEOGSG,2004-01-01,2025-07-05,None,199299399,3.630,3629732.16
4,1005,"LINESTRING (335333.2 7396337.369, 335534.914 7...",79,8583448.0,25.0,8,BELEM,BEL,1:5000,GEOGSG,2004-01-01,2025-07-05,None,199299399,6.047,6046777.42
5,1006,"LINESTRING (339329.18 7395232.734, 339530.935 ...",73,8583424.0,25.0,80,TATUAPE,TAT,1:5000,GEOGSG,2004-01-01,2025-07-05,None,199299399,8.514,8514313.05


In [8]:
# ônibus por distrito
onibus_por_distrito = trajetos_distritos.groupby("nm_distrit")["id_onibus"].nunique().reset_index()

# distância percorrida por distrito
distancia_por_onibus = df_final.groupby("id_onibus")["distancia_percorrida"].sum().reset_index()
trajetos_distritos = trajetos_distritos.merge(distancia_por_onibus, on="id_onibus")
dist_por_distrito = trajetos_distritos.groupby("nm_distrit")["distancia_percorrida"].sum().reset_index()